# 13.0 Dynamic Programming

## 13.1 Number of ways to climb a staircase

### Problem Statement
Implement a function that given some height of a staircase, named $h$, and set of possible steps, named $S$, returns all of the unique ways to climb the staircase.

In [1]:
from collections import namedtuple
import unittest


def staircase(h, S, cache={}):
    """Return unique ways to climb staircase height h using steps S."""

    soltns = set()
    for s in S:
        if s == h:
            soltns.add((s,))
        elif h-s > 0:
            subs = (staircase(h-s, S, cache=cache) 
                    if h-s not in cache else cache[h-s])
            for sub in subs:
                soltns.add((s,) + sub)
                soltns.add(sub + (s,))

    return soltns


class StaircaseTest(unittest.TestCase):

    def test_staircase(self):
        case = namedtuple('case', ['h','S','expected'])
        cases = [
            case(2, {1}, {(1,1)}),
            case(2, {1,2}, {(1,1),(2,)}),
            case(3, {1}, {(1,1,1)}),
            case(3, {1,2}, {(1,1,1),(1,2),(2,1)}),
            case(3, {1,2,3}, {(1,1,1),(1,2),(2,1),(3,)}),
            case(4, {1,2}, 
                 {(1,1,1,1),(2,1,1),(1,2,1),(1,1,2),(2,2)}),
            case(4, {1,2,3}, 
                 {(1,1,1,1),(2,1,1),(1,2,1),(1,1,2),(2,2),(1,3),(3,1)}),
            case(5, {1,2,3},
                 {(1,1,1,1,1),
                  (2,1,1,1),(1,2,1,1),(1,1,2,1),(1,1,1,2),
                  (2,2,1),(1,2,2),(2,1,2),
                  (3,1,1),(1,3,1),(1,1,3),(3,2),(2,3)}),
        ]
        for c in cases:
            rcv = staircase(c.h, c.S)
            self.assertEqual(rcv, c.expected)


unittest.main(StaircaseTest(), argv=[''], verbosity=2, exit=False)

test_staircase (__main__.StaircaseTest) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK
